In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init 
import torch.optim as optim 
import torch.nn.functional as F 

from torchtext import data 
from torchtext import vocab 
from torchtext import datasets 

import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
# Hyper Params 
batch_size = 32
output_size = 2 
hidden_size = 256 
embedding_length = 300 

In [ ]:
'''
Step 1 : テキストの読み込み
Step 2 : テキストの分割とトークン化
・ Field クラスが、tochvision でいう transforms のイメージ
'''

tokenize = lambda x: x.split()
TEXT = data.Field(
    sequential=True, # テキストが可変長の場合に True （パディングの作成対象になる）
    tokenize=tokenize, # トークン化の方法を記載した関数
    lower=True, # 大文字を小文字に変換する場合 True 
    include_lengths=True, # イテレータに含まれる 1 テキストとの単語数を表示（イテレータが長さも含めたタプルを返す）
    batch_first=True, # Tensor の1次元めをバッチサイズの次元にする
    fix_length=200 # 1文の中で難単語まで使用するかを指定
    )

LABEL = data.LabelField()

# データのダウンロード
train_dataset, test_dataset = datasets.IMDB.splits(TEXT, LABEL)
train_dataset, val_dataset = train_dataset.split()

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.2MB/s]


In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

(17500, 7500, 25000)

In [ ]:
'''
Step 3 : トークンのインデックス化
Step 4 : 複数テキストのバッチ化

TEXT.build_vocab : 辞書を作成
TEXT.vocab.freqs : コーパス中の単語毎の出現回数を表示
TEXT.vocab.itos : index から string（単語） の変換
TEXT.vocab.stoi : 逆
TEXT.vocab.vectors : 学習済み埋め込みベクトルの指定
'''

# 辞書を作成
TEXT.build_vocab(
    train_dataset,
    min_freq=3, # 出現頻度の低い単語を省く
    vectors=vocab.GloVe(name="6B", dim=300) # 学習済み埋め込みベクトルを適用
    )
LABEL.build_vocab(train_dataset)

print("単語の件数の Top 10")
print(TEXT.vocab.freqs.most_common(10))

print("")
print("ラベルごと件数")
print(LABEL.vocab.freqs)

# テキストのバッチ化
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_dataset, val_dataset, test_dataset),
    batch_size=32,
    sort_key=lambda x: len(x.text),
    repeat=False,
    shuffle=True
)

# 単語数
vocab_size = len(TEXT.vocab)

# 埋め込みベクトル
word_embeddings = TEXT.vocab.vectors 

.vector_cache/glove.6B.zip: 862MB [06:26, 2.23MB/s]                           
100%|█████████▉| 399794/400000 [00:38<00:00, 10119.64it/s]

単語の件数の Top 10
[('the', 225396), ('a', 111558), ('and', 110886), ('of', 101230), ('to', 93557), ('is', 73319), ('in', 63226), ('i', 49185), ('this', 48605), ('that', 46453)]

ラベルごと件数
Counter({'pos': 8752, 'neg': 8748})


In [ ]:
print(vocab_size)
print(TEXT.vocab.vectors.size())

55508
torch.Size([55508, 300])


In [ ]:
# データの確認

for i, batch in enumerate(train_iter):
  print("# (batch_size, seq_length) -> fix_length=200 としたので、seq_length=200")
  print(batch.text[0].size())
  print("")
  print("# 200 単語に満たない文の場合、<pad>, すなわち 1 で200単語になるまでパディングされる")
  print(batch.text[0][0])

  print("")
  print("# ラベルのサイズ")
  print(batch.text[1].size())

  print("")
  print("# 1 データ目の単語列（数字）")
  print("# text[0][1] testのバッチの、2サンプル目")
  print(batch.text[0][1])

  print("")
  print("# 1 データ目の単語（文字に逆変換）")
  print([TEXT.vocab.itos[data] for data in batch.text[0][1].tolist()])

  break

# (batch_size, seq_length) -> fix_length=200 としたので、seq_length=200
torch.Size([32, 200])

# 200 単語に満たない文の場合、<pad>, すなわち 1 で200単語になるまでパディングされる
tensor([  207,    10,    29,     2, 24059,    25,  1980,   106,     2,   199,
            4,    21,   355,   983,  1251,  3853,   494,     3, 20710, 21320,
           13,   165,   236,    12,   260,     6,   245,    10,     7,     3,
           84,    87,   199,     4,    35,  4513,     6,  4876, 14985,    16,
          295,   301,   591,     3,  3823,     4,   161,    47,    19,     3,
        20447,    16,     3,   277,    27, 50538,    13, 25446,  2819,     5,
          604,     0,    12,   617,     2,    77,     5,     2,   182,   199,
           19,     2,  1555,     5,     2,  4694,     8, 23650,   247,    82,
          544,    10,     6,    28,     3,    20,    43,     2, 11665,    44,
          955, 11265,     7,  1731, 25610,   100,  2446,     6,     2,  1015,
          573,   115,     5,    25,   673,   941,    86,     0, 54362,     0,
 

In [ ]:
class LstmClassifier(nn.Module):
  def __init__(self, batch_size, hidden_size, output_size, vocab_size, embedding_length, weights):
    super().__init__()

    self.batch_size = batch_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.vocab_size = vocab_size
    self.embed = nn.Embedding(vocab_size, embedding_length)

    # 学習済み埋め込みベクトルを使用
    self.embed.weight.data.copy_(weights)
    self.lstm = nn.LSTM(embedding_length, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    x = self.embed(x)

    h0 = torch.zeros(1, self.batch_size, self.hidden_size).to(device)
    c0 = torch.zeros(1, self.batch_size, self.hidden_size).to(device) 

    x, (h, c) = self.lstm(x, (h0, c0))

    out = self.fc(x.view(x.size(0), -1))

    return out 

device = "cuda" if torch.cuda.is_available else "cpu"

net = LstmClassifier(batch_size, hidden_size, output_size, vocab_size, embedding_length, word_embeddings)
net.to(device)

criterion = nn.CrossEntropyLoss()
optim = optim.Adam(net.parameters(), lr=0.01)

In [ ]:
# 学習ループ

num_epochs = 10

train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

for epoch in range(num_epochs):
  train_loss = 0
  train_acc = 0
  val_loss = 0
  val_acc = 0

  net.train()
  for i, batch in enumerate(train_iter):
    text = batch.text[0].to(device)
    #if (text.size()[0] is not 32):
    #  continue
    labels = batch.label.to(device)

    optim.zero_grad()
    outputs = net(text)
    loss = criterion(outputs, labels)
    train_loss += loss.item()
    train_acc += (outputs.max(1)[1] == labels).sum().item()

    loss.backward()
    optim.step()

In [ ]:
# テストデータで推論

net.eval()

with torch.no_grad():
  total = 0
  test_acc = 0
  for batch in test_iter:
    text = batch.text[0].to(device)
    if (text.size()[0] is not 32):
      continue 
    labels = batch.label.to(device)

    outputs = net(text)
    test_acc +=  (outputs.max(1)[1] == labels).sum().item()
    total += labels.size(0)

  print("精度 : {} %".format(100 * test_acc / total))

精度 : 74.90396927016646 %
